1. Fine-tine SumBERT with GRUEN metric
   - investigate other metrics to use
   - 

In [ ]:
!python3 -m pip install transformers sentencepiece datasets rouge_score torch==1.10.0 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!python3 -m pip install wmd==1.3.1
!python -m spacy download en_core_web_md

from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from transformers import *
from datasets import load_metric, Dataset
import torch as pt

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 96.4 MB 522 kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# cd /content/drive/My Drive/W266-final-project/

In [ ]:
logging.set_verbosity_warning()
GPU = 1
model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-xsum-12-6").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6", framework="pt")

## Load in the data over here

In [ ]:
base = "/content/drive/MyDrive/"
data = pd.read_csv(base + "W266-final-project/fine_tuned_data_masked.csv")
summaries = pd.read_csv(base + "W266-final-project/bart-baseline.csv")

In [ ]:
def get_summary(tp, num):
    path = base + "W266-final-project/data/BBC News Summary/Summaries/{0}/{1}.txt".format(
       tp, str(num).zfill(3))
    ret = None
    with open(path) as f:
       ret = f.read()
    return ret
  

In [ ]:
# stitch article summaries from data together, get one list with all the masked summaries available
# problem: the summaries for the fine tuning data hasn' actually been calculated yet. 
# new_data = {'type':[], 'original':[], 'masked':[], 'label':[]}
new_data = {'article': [], 'summary': []} # []
cols = ['business', 'entertainment', 'politics', 'sport', 'tech']
cols_masked = [c+"_masked" for c in cols]
lbl = 0
for i, row in data.iterrows():
  for col, col_mask in zip(cols, cols_masked):
    masked = row[col_mask].replace("<mask>", " <mask>")
    original = row[col + "original"]
    # print((summaries[col] == row[col]).unique())
    summary = get_summary(col, int(row[col]))
    new_data['article'].append(original)
    new_data['summary'].append(summary)

    lbl += 1
  
  if i > 1:
    break

print(new_data['summary'])

["The UK's biggest brewer, Scottish and Newcastle (S&N), is to buy 37.5% of India's United Breweries in a deal worth 4.66bn rupees ($106m:£54.6m).S&N will buy a 17.5% equity stake in United, maker of the well-known Kingfisher lager brand, and make a public offer to buy another 20% stake.In 2002 S&N and United agreed to form a strategic partnership, one that would include a joint venture business and a UK investment in the Indian brewer.Millennium Alcobev will now be merged with United, which expects post-merger to have about half of India's beer market.In addition to the equity stake S&N is to invest 2.47bn rupees in United through non-convertible redeemable preference shares.Meanwhile, United's budget airline, Kingfisher Airlines, is to buy 10 A320 aircraft from Airbus and has the option to buy 20 more aircraft in a deal worth up to $1.8bn.", "Original Abba member Bjorn Ulvaeus set about translating the musical into the band's mother tongue for the celebrated homecoming, alongside Swe

## GRUEN Stuff


In [ ]:
import en_core_web_md
import difflib
import editdistance
import math
import numpy as np
import re
import spacy
import string
import torch
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer, BertForMaskedLM
from transformers import glue_convert_examples_to_features
from transformers.data.processors.utils import InputExample
from wmd import WMD
import nltk
nltk.download('punkt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


""" Processing """
def preprocess_candidates(candidates):
    for i in range(len(candidates)):
        candidates[i] = candidates[i].strip()
        candidates[i] = '. '.join(candidates[i].split('\n\n'))
        candidates[i] = '. '.join(candidates[i].split('\n'))
        candidates[i] = '.'.join(candidates[i].split('..'))
        candidates[i] = '. '.join(candidates[i].split('.'))
        candidates[i] = '. '.join(candidates[i].split('. . '))
        candidates[i] = '. '.join(candidates[i].split('.  . '))
        while len(candidates[i].split('  ')) > 1:
            candidates[i] = ' '.join(candidates[i].split('  '))
        myre = re.search(r'(\d+)\. (\d+)', candidates[i])
        while myre:
            candidates[i] = 'UNK'.join(candidates[i].split(myre.group()))
            myre = re.search(r'(\d+)\. (\d+)', candidates[i])
        candidates[i] = candidates[i].strip()
    processed_candidates = []
    for candidate_i in candidates:
        sentences = sent_tokenize(candidate_i)
        out_i = []
        for sentence_i in sentences:
            if len(sentence_i.translate(str.maketrans('', '', string.punctuation)).split()) > 1:  # More than one word.
                out_i.append(sentence_i)
        processed_candidates.append(out_i)
    return processed_candidates


""" Scores Calculation """
def get_lm_score(sentences):
    def score_sentence(sentence, tokenizer, model):
        # if len(sentence.strip().split()) <= 1:
        #     return 10000
        tokenize_input = tokenizer.tokenize(sentence)
        if len(tokenize_input) > 510:
            tokenize_input = tokenize_input[:510]
        input_ids = torch.tensor(tokenizer.encode(tokenize_input)).unsqueeze(0).to(device)
        with torch.no_grad():
            loss = model(input_ids, labels=input_ids)[0]
        return math.exp(loss.item())

    model_name = 'bert-base-cased'
    model = BertForMaskedLM.from_pretrained(model_name).to(device)
    model.eval()
    tokenizer = BertTokenizer.from_pretrained(model_name)
    lm_score = []
    for sentence in tqdm(sentences):
        if len(sentence) == 0:
            lm_score.append(0.0)
            continue
        score_i = 0.0
        for x in sentence:
            score_i += score_sentence(x, tokenizer, model)
        score_i /= len(sentence)
        lm_score.append(score_i)
    return lm_score


def get_cola_score(sentences):
    def load_pretrained_cola_model(model_name, saved_pretrained_CoLA_model_dir):
        config_class, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = config_class.from_pretrained(saved_pretrained_CoLA_model_dir, num_labels=2, finetuning_task='CoLA')
        tokenizer = tokenizer_class.from_pretrained(saved_pretrained_CoLA_model_dir, do_lower_case=0)
        model = model_class.from_pretrained(saved_pretrained_CoLA_model_dir, from_tf=bool('.ckpt' in model_name), config=config).to(device)
        model.eval()
        return tokenizer, model

    def evaluate_cola(model, candidates, tokenizer, model_name):

        def load_and_cache_examples(candidates, tokenizer):
            max_length = 128
            examples = [InputExample(guid=str(i), text_a=x) for i,x in enumerate(candidates)]
            features = glue_convert_examples_to_features(examples, tokenizer, label_list=["0", "1"], max_length=max_length, output_mode="classification")
            # Convert to Tensors and build dataset
            all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
            all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
            all_labels = torch.tensor([0 for f in features], dtype=torch.long)
            all_token_type_ids = torch.tensor([[0.0]*max_length for f in features], dtype=torch.long)
            dataset = torch.utils.data.TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
            return dataset

        eval_dataset = load_and_cache_examples(candidates, tokenizer)
        eval_dataloader = torch.utils.data.DataLoader(eval_dataset, sampler=torch.utils.data.SequentialSampler(eval_dataset), batch_size=max(1, torch.cuda.device_count()))
        preds = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[3]}
                if model_name.split('-')[0] != 'distilbert':
                    inputs['token_type_ids'] = batch[2] if model_name.split('-')[0] in ['bert', 'xlnet'] else None  # XLM, DistilBERT and RoBERTa don't use segment_ids
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

            if preds is None:
                preds = logits.detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        return preds[:, 1].tolist()

    def convert_sentence_score_to_paragraph_score(sentence_score, sent_length):
        paragraph_score = []
        pointer = 0
        for i in sent_length:
            if i == 0:
                paragraph_score.append(0.0)
                continue
            temp_a = sentence_score[pointer:pointer + i]
            paragraph_score.append(sum(temp_a) / len(temp_a))
            pointer += i
        return paragraph_score

    model_name = 'bert-base-cased'
    saved_pretrained_CoLA_model_dir = base + '/W266-final-project/cola_model/bert-base-cased/'
    tokenizer, model = load_pretrained_cola_model(model_name, saved_pretrained_CoLA_model_dir)
    candidates = [y for x in sentences for y in x]
    sent_length = [len(x) for x in sentences]
    cola_score = evaluate_cola(model, candidates, tokenizer, model_name)
    cola_score = convert_sentence_score_to_paragraph_score(cola_score, sent_length)
    return cola_score


def get_grammaticality_score(processed_candidates):
    lm_score = get_lm_score(processed_candidates)
    cola_score = get_cola_score(processed_candidates)
    grammaticality_score = [1.0 * math.exp(-0.5*x) + 1.0 * y for x, y in zip(lm_score, cola_score)]
    grammaticality_score = [max(0, x / 8.0 + 0.5) for x in grammaticality_score]  # re-scale
    return grammaticality_score


def get_redundancy_score(all_summary):
    def if_two_sentence_redundant(a, b):
        """ Determine whether there is redundancy between two sentences. """
        if a == b:
            return 4
        if (a in b) or (b in a):
            return 4
        flag_num = 0
        a_split = a.split()
        b_split = b.split()
        if max(len(a_split), len(b_split)) >= 5:
            longest_common_substring = difflib.SequenceMatcher(None, a, b).find_longest_match(0, len(a), 0, len(b))
            LCS_string_length = longest_common_substring.size
            if LCS_string_length > 0.8 * min(len(a), len(b)):
                flag_num += 1
            LCS_word_length = len(a[longest_common_substring[0]: (longest_common_substring[0]+LCS_string_length)].strip().split())
            if LCS_word_length > 0.8 * min(len(a_split), len(b_split)):
                flag_num += 1
            edit_distance = editdistance.eval(a, b)
            if edit_distance < 0.6 * max(len(a), len(b)):  # Number of modifications from the longer sentence is too small.
                flag_num += 1
            number_of_common_word = len([x for x in a_split if x in b_split])
            if number_of_common_word > 0.8 * min(len(a_split), len(b_split)):
                flag_num += 1
        return flag_num

    redundancy_score = [0.0 for x in range(len(all_summary))]
    for i in range(len(all_summary)):
        flag = 0
        summary = all_summary[i]
        if len(summary) == 1:
            continue
        for j in range(len(summary) - 1):  # for pairwise redundancy
            for k in range(j + 1, len(summary)):
                flag += if_two_sentence_redundant(summary[j].strip(), summary[k].strip())
        redundancy_score[i] += -0.1 * flag
    return redundancy_score


def get_focus_score(all_summary):
    def compute_sentence_similarity():
        nlp = en_core_web_md.load()
        nlp.add_pipe(WMD.SpacySimilarityHook(nlp), last=True)
        all_score = []
        for i in range(len(all_summary)):
            if len(all_summary[i]) == 1:
                all_score.append([1.0])
                continue
            score = []
            for j in range(1, len(all_summary[i])):
                doc1 = nlp(all_summary[i][j-1])
                doc2 = nlp(all_summary[i][j])
                try:
                    score.append(1.0/(1.0 + math.exp(-doc1.similarity(doc2)+7)))
                except:
                    score.append(1.0)
            all_score.append(score)
        return all_score

    all_score = compute_sentence_similarity()
    focus_score = [0.0 for x in range(len(all_summary))]
    for i in range(len(all_score)):
        if len(all_score[i]) == 0:
            continue
        if min(all_score[i]) < 0.05:
            focus_score[i] -= 0.1
    return focus_score


def get_gruen(candidates):
    processed_candidates = preprocess_candidates(candidates)
    grammaticality_score = get_grammaticality_score(processed_candidates)
    redundancy_score = get_redundancy_score(processed_candidates)
    focus_score = get_focus_score(processed_candidates)
    gruen_score = [min(1, max(0, sum(i))) for i in zip(grammaticality_score, redundancy_score, focus_score)]
    return gruen_score


def score_model():
  import pandas as pd
  data = pd.read_csv(base + "/W266-final-project/bart-baseline.csv")

  scores = {}
  for col in ['business-results', 'sport-results', 'tech-results', 'entertainment-results']:
    gruen_score = get_gruen(list(data[col]))
    print(col + " " + str(np.mean(gruen_score)))
    scores[col] = gruen_score

# score_model()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Training

In [ ]:
L = 784

def tokenize_function(ex):
  print(ex)
  print("***")
  tokens = tokenizer(ex['article'], padding='max_length', truncation=True, max_length=L, return_tensors='pt')
  label = tokenizer(ex['summary'], padding='max_length', truncation=True, max_length=L, return_tensors='pt')
  print(tokens['attention_mask'].size())
  tokens['attention_mask'] = tokens['attention_mask'].flatten().to('cuda')
  tokens['input_ids'] = tokens['input_ids'].flatten().to('cuda')
  # print(label)
  tokens['labels'] = label['input_ids'].flatten().to('cuda')
  # tokens['target'] = 
  return tokens

# dataset = Dataset.from_dict(new_data)

print('Tokenizing data. This may take a while...')
tokenized_dataset = Dataset.from_dict(new_data).map(tokenize_function, writer_batch_size=1).remove_columns(['article', 'summary'])
partition = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = partition['train']
eval_dataset = partition['test']
print(pt.Tensor(train_dataset[0]['input_ids']).size())

Tokenizing data. This may take a while...


  0%|          | 0/15 [00:00<?, ?ex/s]

{'article': 'S&N extends Indian beer venture\n\nThe UK\'s biggest brewer, Scottish and Newcastle (S&N), is to buy 37.5% of India\'s United Breweries in a deal worth 4.66bn rupees ($106m:£54.6m).\n\nS&N will buy a 17.5% equity stake in United, maker of the well-known Kingfisher lager brand, and make a public offer to buy another 20% stake. A similar holding will be controlled by Vijay Mallya, chair of the Indian firm. The deal was a "natural development" of its joint venture with United, said Tony Froggatt, S&N\'s chief executive.\n\nIts top brands include Newcastle Brown Ale, Foster\'s, John Smith\'s, Strongbow and Kronenbourg. In 2002 S&N and United agreed to form a strategic partnership, one that would include a joint venture business and a UK investment in the Indian brewer. The joint venture was established in May 2003. with both parties having a 40% stake in the venture - Millennium Alcobev. Millennium Alcobev will now be merged with United, which expects post-merger to have about

In [ ]:
# plan: get sample summaries done, fine tune model with rouge 

metric = load_metric("rouge")
# print(metric.inputs_description)

def compute_metrics(eval_pred):
  print(eval_pred)
  print("test123")
  return metric.compute(eval_pred)

training_args = TrainingArguments(
    num_train_epochs=4,           
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,   
    warmup_steps=500,               
    weight_decay=0.01,              
    logging_dir='./logs',
    eval_accumulation_steps=1,
    output_dir="./out")

model.gradient_checkpointing_enable()
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=eval_dataset,
  data_collator=DefaultDataCollator(return_tensors='pt')
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 48


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=48, training_loss=9.221689860026041, metrics={'train_runtime': 22.3501, 'train_samples_per_second': 2.148, 'train_steps_per_second': 2.148, 'total_flos': 56885769142272.0, 'train_loss': 9.221689860026041, 'epoch': 4.0})

In [ ]:
model.save_pretrained('./experiment-1')

Configuration saved in ./experiment-1/config.json
Model weights saved in ./experiment-1/pytorch_model.bin


# GRUEN Metric Calculation

In [ ]:
!python3 -m pip install transformers sentencepiece datasets rouge_score torch==1.10.0 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!python3 -m pip install wmd==1.3.1
!python -m spacy download en_core_web_md

from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from transformers import *
from datasets import load_metric, Dataset
import torch as pt

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 96.4 MB 532 kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
logging.set_verbosity_warning()
model = BartForConditionalGeneration.from_pretrained("./experiment-1/")
base = "/content/drive/MyDrive/"

In [ ]:


data_test = pd.read_csv(base + "W266-final-project/test_df_indicies.csv")

def assemble_dataset(df, masked=False):
  # go thru all columns, get article type, access approppriate article from masked dataset

  ret = {'type': [], 'article': [] } #'summary': []}
  select_types = ['business', 'entertainment', 'politics', 'sport', 'tech']
  # cols_masked = [c+"_masked" for c in cols]
  lbl = 0
  for col in df:
    lbl += 1
    article_type = col[:-2]

    if article_type in select_types:
      data = pd.read_csv(base + "W266-final-project/{0}_masked_articles.csv".format(article_type))
      for idx in df[col]:
        flag = data[article_type + " indicies"] == idx
        row = data[flag]
        try:
          flag = " masked" if masked else " original"
          article = row[article_type + flag].iloc[0].replace("<mask>", " <mask>")
        except Exception as e:
          
          print("MISSING ENTRY:\n [[{0}]]\n[[{1}]]".format(idx, row))
        ret['article'].append(article)
        # ret['summary'].append(get_summary(article_type, int(idx)))
        ret['type'].append(article_type)
    if lbl % 3 == 0:
      print("{0} progress marker".format(col))
    # for j, col in row.iterrows():
    #   masked = row[col_mask].replace("<mask>", " <mask>")
    #   original = row[col + "original"]
    #   # print((summaries[col] == row[col]).unique())
    #   summary = get_summary(col, int(row[col]))
    #   new_data['article'].append(original)
    #   new_data['summary'].append(summary)

    #   lbl += 1
    
    # if i > 1:
    #   break

  return ret

test_articles = assemble_dataset(data_test)

entertainment-0 progress marker
MISSING ENTRY:
 [[401.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, tech indicies, tech masked, tech original]
Index: []]]
tech-0 progress marker
politics-1 progress marker
MISSING ENTRY:
 [[510.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, business indicies, business masked, business original]
Index: []]]
business-2 progress marker
MISSING ENTRY:
 [[386.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, entertainment indicies, entertainment masked, entertainment original]
Index: []]]
sport-2 progress marker
entertainment-3 progress marker
MISSING ENTRY:
 [[511.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, sport indicies, sport masked, sport original]
Index: []]]
tech-3 progress marker
MISSING ENTRY:
 [[386.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, entertainment indicies, entertainment masked, entertainment original]
Index: []]]
MISSING ENTRY:
 [[417.0]]
[[Empty DataFrame
Columns: [Unnamed: 0, politics indicies, politics masked, politics original]
Index: []]]
polit

In [ ]:
sifted_test_articles = {i : [] for i in test_articles}
for i in range(500):
  test_tokenizer = tokenizer(test_articles['article'][i])
  if len(test_tokenizer['input_ids']) <= 1024:
    for c in test_articles:
      sifted_test_articles[c].append(test_articles[c][i])


Token indices sequence length is longer than the specified maximum sequence length for this model (3105 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


['business',
 'business',
 'business',
 'business',
 'business',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'entertainment',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'sport',
 'sport',
 'sport',
 'sport',
 'sport',
 'tech',
 'tech',
 'tech',
 'tech',
 'tech']

In [ ]:
new_summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)
select_types = ['business', 'entertainment', 'politics', 'sport', 'tech']
gruen_scores = {i: [] for i in select_types}
summs = new_summarizer(sifted_test_articles['article'][::20])
summs

# eval_data = new_data['article'][-1:-10] # Change this line to be whatever evaluation dataset we have 
# for tp, article in zip(test_articles['type'], test_articles['article']): 
#   print("***")
#   summ = new_summarizer(article, min_length=30, max_length=130)
#   sc = get_gruen([summ[0]['summary_text']])
#   gruen_scores[tp].append(sc)

#   if len(gruen_scores[tp]) > 50:
#     break

[{'summary_text': 'Wimbledon and Hall Green greyhound tracks have been put up for sale by gaming group Wembley.'},
 {'summary_text': ' The BBC News website looks at some of the key points of the US airline chaos over the Christmas weekend.'},
 {'summary_text': "More than two-thirds of companies reporting on social and environmental issues don't look at their supply chains, suggests a survey."},
 {'summary_text': 'Former Worldcom boss Bernie Ebbers never made accounting decisions, a witness at his fraud trial has told jurors.'},
 {'summary_text': " A US company has pulled out of a major contract to rebuild Iraq's transport system after attacks on reconstruction efforts, Pentagon officials have said."},
 {'summary_text': 'French actress Audrey Tautou will play the female lead in the film adaptation of The Da Vinci Code, according to reports.'},
 {'summary_text': 'Celebrity Big Brother contestant Germaine Greer has criticised the show\'s "bullying" of housemates, saying it could encourage

Ideas for the Future:
 - show how the GRUEN metric is a proxy for readabillity; show an unreadable summary and GRUEN score, readable summary and GRUEN score

In [ ]:
ext_summs = [a['summary_text'] for a in summs]

In [ ]:
bla = get_gruen(ext_summs)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 25/25 [00:00<00:00, 107.17it/s]
/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/

In [31]:
gruen_totals = {i : 0 for i in set(test_articles['type'])}
for tp, sc in zip(sifted_test_articles['type'][::20], bla):
  gruen_totals[tp] += sc / 5

In [32]:
gruen_totals

{'business': 0.7734685754796737,
 'entertainment': 0.859470314524942,
 'politics': 0.8007642861194826,
 'sport': 0.8400955133354324,
 'tech': 0.838198986425583}